In [ ]:
!pip install fpdf
!pip install datasets

import os
import pandas as pd
from fpdf import FPDF
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Desactivar wandb para evitar que se pida la clave de API
os.environ["WANDB_DISABLED"] = "true"

# Paso 1: Configuración inicial
!pip install transformers datasets -q
!pip install fsspec==2024.10.0 -q  # Instalar versión compatible de fsspec

# Paso 2: Descarga y preparación del dataset
dataset = load_dataset("emotion")  # Dataset de emociones como ejemplo
dataset = dataset.map(lambda x: {'label': x['label'], 'text': x['text']})

# Dividir el dataset en entrenamiento, validación y prueba
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

# Paso 3: Preprocesamiento de los textos
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(data):
    return tokenizer(data['text'], truncation=True, padding=True)

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

# Paso 4: Carga del modelo
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Paso 5: Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Paso 6: Entrenamiento
trainer.train()

# Paso 7: Evaluación
results = trainer.evaluate(test_dataset)
print("Resultados de evaluación:", results)

# Paso 8: Guardar el modelo entrenado
model.save_pretrained("trained_model")
tokenizer.save_pretrained("trained_model")

# Paso 9: Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Paso 10: Función para recopilar y evaluar los datos interactivos
def collect_data_and_evaluate():
    num_humans = int(input("¿Cuántos humanos se van a evaluar? "))

    # Recopilar datos y realizar evaluación por cada humano
    for i in range(num_humans):
        print(f"\nEvaluando a humano {i + 1}:")

        # Recopilar datos básicos del humano
        name = input("Nombre: ")
        age = int(input("Edad: "))
        marital_status = input("Estado civil: ")

        # Encuesta de 5 preguntas
        print("\nResponde a las siguientes preguntas (a, b, c, d):")
        question_1 = input("1. ¿Te sientes estresado últimamente? (a) Sí, mucho (b) Algo (c) No mucho (d) No, nada: ")
        question_2 = input("2. ¿Sueles tener problemas para dormir? (a) Sí, frecuentemente (b) Ocasionalmente (c) Rara vez (d) Nunca: ")
        question_3 = input("3. ¿Te sientes feliz con tu vida actualmente? (a) Sí, totalmente (b) Algo (c) No mucho (d) No, para nada: ")
        question_4 = input("4. ¿Sientes que tienes apoyo emocional? (a) Sí, mucho (b) Algo (c) No mucho (d) No, nada: ")
        question_5 = input("5. ¿Te sientes fatigado/a a menudo? (a) Sí, todo el tiempo (b) A veces (c) Rara vez (d) Nunca: ")

        # Recoger el texto de 2 párrafos sobre cómo se siente
        print("\nEscribe un texto de dos párrafos contando cómo te sientes:")
        text = input("Texto: ")

        # Procesar texto y hacer predicción
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        label = torch.argmax(outputs.logits, dim=1).item()
        labels_map = dataset["train"].features["label"].names

        print(f"\nEvaluación del humano {i + 1} ({name}):")
        print(f"Predicción del modelo: {labels_map[label]}")

        # Guardar datos en un DataFrame para generar el Excel
        data = {
            'Nombre': name,
            'Edad': age,
            'Estado Civil': marital_status,
            'Respuesta 1': question_1,
            'Respuesta 2': question_2,
            'Respuesta 3': question_3,
            'Respuesta 4': question_4,
            'Respuesta 5': question_5,
            'Texto': text,
            'Predicción': labels_map[label],
        }
        df = pd.DataFrame([data])

        # Guardar en archivo Excel
        excel_filename = f"{name}_evaluacion.xlsx"
        df.to_excel(excel_filename, index=False)
        print(f"Resultados guardados en: {excel_filename}")

        # Generar PDF con la evaluación
        pdf_filename = f"{name}_evaluacion.pdf"
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        # Agregar contenido al PDF
        pdf.cell(200, 10, txt=f"Evaluación de {name}", ln=True, align='C')
        pdf.ln(10)
        pdf.multi_cell(0, 10, txt=f"Edad: {age}\nEstado Civil: {marital_status}\n\nRespuestas de la Encuesta:\n1. {question_1}\n2. {question_2}\n3. {question_3}\n4. {question_4}\n5. {question_5}\n\nTexto: {text}\n\nPredicción: {labels_map[label]}")
        pdf.output(pdf_filename)
        print(f"Resultados guardados en: {pdf_filename}")

# Ejecutar la evaluación interactiva
collect_data_and_evaluate()


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=bb55118058d907893d83f9070cc709214a3c649de542b5f2f93660a2a01d0b8a
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-1-78ca2ca9e610>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.278900,0.208806,0.925500,0.930102,0.925500,0.926460
2,0.091400,0.144653,0.938500,0.939838,0.938500,0.937926
3,0.076300,0.148323,0.942500,0.942907,0.942500,0.942498


Resultados de evaluación: {'eval_loss': 0.16070373356342316, 'eval_accuracy': 0.9295, 'eval_precision': 0.9294261263410812, 'eval_recall': 0.9295, 'eval_f1': 0.9293279828921412, 'eval_runtime': 1.8955, 'eval_samples_per_second': 1055.127, 'eval_steps_per_second': 65.945, 'epoch': 3.0}
¿Cuántos humanos se van a evaluar? 3

Evaluando a humano 1:
Nombre: Laura Martínez
Edad: 32
Estado civil: Casada

Responde a las siguientes preguntas (a, b, c, d):
1. ¿Te sientes estresado últimamente? (a) Sí, mucho (b) Algo (c) No mucho (d) No, nada: b
2. ¿Sueles tener problemas para dormir? (a) Sí, frecuentemente (b) Ocasionalmente (c) Rara vez (d) Nunca: c
3. ¿Te sientes feliz con tu vida actualmente? (a) Sí, totalmente (b) Algo (c) No mucho (d) No, para nada: a
4. ¿Sientes que tienes apoyo emocional? (a) Sí, mucho (b) Algo (c) No mucho (d) No, nada: a
5. ¿Te sientes fatigado/a a menudo? (a) Sí, todo el tiempo (b) A veces (c) Rara vez (d) Nunca: b

Escribe un texto de dos párrafos contando cómo te sien